### Carga de las imagenes

In [1]:
from image_services import get_image_urls
from skimage import io
import solar_image_analysis as sia
import re 
from datetime import datetime

Acá esta como ejemplo como podemos generar todos los links para cualquera de las  carpertas dado un rango de fechas, el nombre de la carpeta (tipo de imagen, en que banda se tomo), indicando apropiadamente la fecha de incio y la frecuencia con la que se toman, como se muestra a continuación estan generando para tres casos


In [2]:
# Metodo 1
hmiigr_urls = get_image_urls( image_type = 'hmiigr',start='01/01/2022 00:00:00', end= '01/01/2023', frequency='90T')

# Metodo 2
mdimag_urls =  get_image_urls( image_type = 'eit171', year= '2013')


Acá se estan probando el calculo de los parametros que se van implementando para 10 imagenes

In [26]:
import pandas as pd
from urllib.error import HTTPError

def compute_metrics(image_type:str, year:str = None, start:str = None, end:str = None,  frequency:str=  None ) -> pd.DataFrame:
    
    """
    Computes various image metrics for a set of images within a given date range and specified type and frequency.

    Parameters
    ----------
    start_date : str
        Start date for retrieving the images, formatted as 'YYYY-MM-DD'.
    
    end_date : str
        End date for retrieving the images, formatted as 'YYYY-MM-DD'.
    
    image_type : str
        Type/category of the image to retrieve. 
    
    frequency : str
        Frequency for retrieving images, e.g., 'daily', 'hourly'.

    Returns
    -------
    pd.DataFrame
        DataFrame containing computed metrics for each image, with one row per image 
        and columns for each feature. The index of the DataFrame corresponds to the datetime 
        extracted from the image URL.
        
    Notes
    -----
    The function fetches image URLs based on the provided date range, type, and frequency. 
    For each URL, it reads the image and computes a set of metrics, such as entropy, mean 
    intensity, and fractal dimension. If the image at a particular URL is inaccessible, it 
    skips that URL. The extracted date and time from the image URL is used as the index 
    for the resulting DataFrame.
    """

    image_urls = get_image_urls(image_type, year, start, end,  frequency )

    
    #Create an empty DataFrame to store the results
    df = pd.DataFrame(columns=['entropy', 'mean_intensity', 'std_deviation', 'fractal_dimension', 'skewness', 'kurtosis', 'uniformity', 'relative_smoothness', 
                                'taruma_contrast', 'taruma_directionality'])

    for url in image_urls[:30]:
        try:
            # Try to read the image
            image = io.imread(url)
            

        except HTTPError:
            #print(f"Unable to access image at {url}")
            continue
            
        # Compute the features
        entropy = sia.compute_entropy(image)
        mean_intensity = sia.compute_mean_intensity(image)
        std_deviation = sia.compute_standard_deviation(image)
        fractal_dimension = sia.compute_fractal_dimension(image)
        skewness = sia.compute_skewness(image)
        kurtosis = sia.compute_kurtosis(image)
        uniformity = sia.compute_uniformity(image)
        relative_smoothness = sia.compute_relative_smoothness(image)
        taruma_contrast = sia.compute_taruma_contrast(image)
        taruma_directionality = sia.compute_taruma_directionality(image)
        # Extract the date from the URL
        date_string = re.findall(r"\d{8}_\d{4}", url)[0]
        date = datetime.strptime(date_string, '%Y%m%d_%H%M')

        # Append the results to the DataFrame
        df.loc[date] = [entropy, mean_intensity, std_deviation, fractal_dimension, skewness, kurtosis, uniformity, relative_smoothness,taruma_contrast, taruma_directionality ]


    return df

# Este es el metodo que teniamos solo con una modificación en el orden de los parametros
#data = compute_metrics( image_type = 'hmiigr',start='01/01/2022 00:00:00', end= '01/01/2023', frequency='90T')
    
# Esta es la otra opción, para años anteriores a 2011
data = compute_metrics( image_type = 'eit171', year= '2013')
#print(data)


In [27]:
data

,entropy,mean_intensity,std_deviation,fractal_dimension,skewness,kurtosis,uniformity,relative_smoothness,taruma_contrast,taruma_directionality
2013-01-01 01:00:00,5.742072,42.745728,49.797079,1.294587,1.384063,4.826050,0.116563,0.999597,1673.053346,0.990590
2013-01-01 13:00:00,5.714206,42.127807,49.351979,1.298669,1.430860,5.066126,0.118150,0.999590,1623.454588,0.990707
2013-01-02 01:00:00,5.732694,42.720532,49.795127,1.295441,1.384909,4.834219,0.117750,0.999597,1672.215008,0.990693
2013-01-02 13:00:00,5.728205,42.928288,50.338898,1.298707,1.392241,4.807031,0.118931,0.999606,1711.347392,0.990748
2013-01-03 01:00:00,5.738676,42.944084,50.570720,1.300750,1.381775,4.700880,0.118041,0.999609,1736.814680,0.990664
2013-01-03 13:00:00,5.780974,44.086663,51.430518,1.294221,1.330283,4.439335,0.115514,0.999622,1822.268254,0.990502
2013-01-04 01:00:00,5.757452,43.686279,51.457323,1.285073,1.356756,4.485594,0.117171,0.999622,1819.446868,0.990597
2013-01-04 13:00:00,5.779960,43.783816,52.112068,1.296581,1.396879,4.575684,0.115177,0.999632,1856.789195,0.990389
2013-01-05 01:00:00,5.822026,44.739053,53.021754,1.295359,1.359173,4.394636,0.112911,0.999644,1941.679061,0.990237
2013-01-05 13:00:00,5.850880,45.770027,54.053462,1.289638,1.347788,4.341559,0.111579,0.999658,2024.117010,0.990166
